
## Exploratory Data Analysis
Clean, process, and create new features.

Compare change in open interest and volume.

**Import required packages & check working directory**

In [1]:
from EDA_fun_multithread import delta_open_interest
from EDA_fun_graph import interest_vs_volume, voi_dividends_ts
import multiprocessing
from multiprocessing.pool import Pool
import numpy as np
import os
import pandas as pd
from pathlib import Path

# Ensure working directory path is correct
if os.getcwd()[-3:] == "EDA":
    os.chdir(os.path.dirname(os.getcwd()))
else:
    pass


**User defined parameters**

In [2]:
ticker = "AAPL"
adjusted_options_path = f"data/adjusted_options/{ticker}/"
dividends_path = f"data/dividends/"
save_path = f"data/EDA1/{ticker}"


**Load adjusted options data**

In [3]:
# Initialize DataFrames
complete_df = pd.DataFrame()
incomplete_df = pd.DataFrame()

# Only get directories
for year in next(os.walk(adjusted_options_path))[1]:
    for file in os.listdir(os.path.join(adjusted_options_path, year)):
        # Load
        temp_df = pd.read_csv(os.path.join(os.path.join(adjusted_options_path, year, file)))

        # Convert columns to correct format
        temp_df["date"] = pd.to_datetime(temp_df["date"]).dt.date
        temp_df["expiration date"] = pd.to_datetime(temp_df["expiration date"]).dt.date
        temp_df["year"] = int(year)

        if file.split("_")[-1] == "complete.csv":
            complete_df = complete_df.append(temp_df, ignore_index=True)
        elif file.split("_")[-1] == "incomplete.csv":
            incomplete_df = incomplete_df.append(temp_df, ignore_index=True)
        else:
            continue


## Feature Engineering

Dividend adjustment: Since all features contain priced-in dividends, we remove them by subtracting pre-calculated dividend contributions, as calculated in [eod_price_and_dividends.py](https://github.com/jacktan1/Options-Project/blob/master/src/eod_price_and_dividends.py).

Logic for removing dividends from strike price:
Consider a stock paying \$5 quarterly dividends. It just passed its ex-div date and closed at \$10. Now to price a \$15 call option expiring a day before the next ex-div date. Even though it might seem extremely OTM, in reality, it is ATM (10 + 5 = 15).

In [4]:
for n in [{"df": complete_df, "tag": "complete"}, {"df": incomplete_df, "tag": "incomplete"}]:
    # Unpack
    option_df = n["df"]

    # Adjust strike and data date
    option_df["adj strike"] = (option_df["strike price"] - option_df["exp date div"]).round(6)
    option_df["adj date close"] = (option_df["date close"] - option_df["date div"]).round(6)

    # Days until expiry
    option_df["days till exp"] = np.busday_count(begindates=list(option_df["date"]),
                                                 enddates=list(option_df["expiration date"]))

    if n["tag"] == "complete":
        option_df["adj exp close"] = (option_df["exp date close"] - option_df["exp date div"]).round(6)
        complete_df = option_df
    elif n["tag"] == "incomplete":
        incomplete_df = option_df


In [5]:
complete_df.head()

,date,expiration date,tag,strike price,ask price,ask size,bid price,bid size,last price,volume,open interest,date div,exp date div,date close,exp date close,year,adj strike,adj date close,days till exp,adj exp close
0,2005-01-03,2005-01-21,call,0.089286,1.044643,0.0,1.039286,0.0,1.044643,560.0,48216.0,0.0,0.0,1.13018,1.25875,2005,0.089286,1.13018,14,1.25875
1,2005-01-03,2005-01-21,put,0.089286,0.000893,0.0,0.000000,0.0,0.000000,0.0,65688.0,0.0,0.0,1.13018,1.25875,2005,0.089286,1.13018,14,1.25875
2,2005-01-03,2005-01-21,call,0.178571,0.955357,0.0,0.950000,0.0,0.948214,0.0,68320.0,0.0,0.0,1.13018,1.25875,2005,0.178571,1.13018,14,1.25875
3,2005-01-03,2005-01-21,put,0.178571,0.000893,0.0,0.000000,0.0,0.000893,0.0,57400.0,0.0,0.0,1.13018,1.25875,2005,0.178571,1.13018,14,1.25875
4,2005-01-03,2005-01-21,call,0.267857,0.866071,0.0,0.860714,0.0,0.891071,560.0,487704.0,0.0,0.0,1.13018,1.25875,2005,0.267857,1.13018,14,1.25875


*Definition:*

**breakeven** - the price movement a ticker must undergo for a listed option to accumulate no gain or loss at expiry. In other words:

Calls: $ breakeven = adj~strike + ask/bid~price - adj~date~close$

Puts: $ breakeven = adj~strike - ask/bid~price - adj~closing$

*Example:* Ticker `ABC` closes at \$100. For a \$90 call option priced at \$15 to "breakeven", it will need to move: \$90 + \$15 - \$100 = \$5.

However, since the theta (time premium) for near-expiry options are so low already, the inferred difference in priced-in dividends may be greater than the theta. As a result, removing dividends will cause the option to be profitable at purchase.
   - This occurs more if "bid price" is used, as "bid theta" is lower than "ask theta".

*Example:* Say we have a high yield ticker `ABC` with an option expiring in 5 days. The theoretical amount of priced-in dividend today is \$9, with the amount increasing to \$10 on date of expiry. However, the theta premium for an ATM call option today may only cost \$0.7. This results in a net profit of \$0.3 per share at purchase if we remove dividends.

**moneyness** - How in the money or out of the money an option is.

Only ask price will be used.

In [6]:
for n in [{"df": complete_df, "tag": "complete"}, {"df": incomplete_df, "tag": "incomplete"}]:
    # Unpack
    temp_df = n["df"]

    calls_df = temp_df[temp_df["tag"] == "call"].copy()
    puts_df = temp_df[temp_df["tag"] == "put"].copy()

    calls_df["adj breakeven"] = (calls_df["adj strike"] + calls_df["ask price"] -
                                 calls_df["adj date close"]).round(6)
    calls_df["raw breakeven"] = (calls_df["strike price"] + calls_df["ask price"] -
                                 calls_df["date close"]).round(6)

    puts_df["adj breakeven"] = (puts_df["adj strike"] - puts_df["ask price"] -
                                puts_df["adj date close"]).round(6)
    puts_df["raw breakeven"] = (puts_df["strike price"] - puts_df["ask price"] -
                                puts_df["date close"]).round(6)

    calls_df["adj moneyness"] = (calls_df["adj date close"] - calls_df["adj strike"]).round(6)

    puts_df["adj moneyness"] = (puts_df["adj strike"] - puts_df["adj date close"]).round(6)

    combined_df = calls_df.append(puts_df)

    if n["tag"] == "complete":
        # Sanity check
        assert complete_df.shape[0] == combined_df.shape[0], f"Number of rows shouldn't change! (type: {n['tag']})"
        complete_df = combined_df
    elif n["tag"] == "incomplete":
        # Sanity check
        assert incomplete_df.shape[0] == combined_df.shape[0], f"Number of rows shouldn't change! (type: {n['tag']})"
        incomplete_df = combined_df


**"open interest" vs. "volume"**

By definition, open interest is the number of open contracts recorded at closing. Volume, on the other hand, is the number of contracts that was traded within that day. Therefore, the volume of an option on day `n+1` should be greater or equal to the change in open interest from day `n` and day `n+1` (unless the option has been exercised).

To do this, we need to compare the relationship between absolute change in open interest and the volume recorded on neighbouring days, to see if the above is true.

First, we calculate the change in open interest between neighbouring days. The returned DataFrame differs in two ways:
   1. Filters out options that expire that very day (there is no tomorrow to observe change)
   2. Filters out options of the last day in entire dataset (no tomorrow to compare with)


In [7]:
voi_complete = pd.DataFrame()
voi_incomplete = pd.DataFrame()
my_pool = Pool(multiprocessing.cpu_count())

for n in [{"df": complete_df, "tag": "complete"}, {"df": incomplete_df, "tag": "incomplete"}]:
    # Bookkeeping variables
    agg_df = pd.DataFrame()
    input_list = []

    # Unpack
    options_df = n["df"]
    year_list = sorted(set(options_df["year"]))

    for year in year_list:
        print(f"Processing -> (year: {year}, type: {n['tag']})")
        # Bookkeeping variables
        agg_year_df = pd.DataFrame()
        input_list = []

        options_year_df = options_df[options_df["year"] == year].copy()
        data_dates = sorted(set(options_year_df["date"]))

        for date_1 in data_dates:
            # Find the following date
            try:
                date_2 = np.min([n for n in data_dates if n > date_1])
            except ValueError:
                # Get first date of next year
                if year < np.max(year_list):
                    date_2 = np.min(options_df[options_df["year"] == (year + 1)]["date"])
                # If no next year, just skip
                else:
                    continue

            # Sanity check
            if np.busday_count(date_1, date_2) > 2:
                print(f"WARNING: Gap between {date_1} and {date_2} is greater than 2 business days!")

            # Get option spreads for the two dates
            if date_1.year == date_2.year:
                input_options_df = options_year_df[options_year_df["date"].isin([date_1, date_2])][
                    ["year", "date", "expiration date", "tag",
                     "adj strike", "open interest", "volume"]]
            else:
                input_options_df = options_df[options_df["date"].isin([date_1, date_2])][
                    ["year", "date", "expiration date", "tag",
                     "adj strike", "open interest", "volume"]]

            input_list.append({"df": input_options_df, "former date": date_1, "latter date": date_2, "year": year})

        # Finished aggregating date pairs for year
        results_list = my_pool.map(delta_open_interest, input_list)

        for result in results_list:
            for msg in result["messages"]:
                print(msg)

        # Append daily results into year df
        agg_year_df = pd.concat([n["df"] for n in results_list], axis=0, ignore_index=True)

        # Append year results into complete df
        agg_df = agg_df.append(agg_year_df)

    # Sort
    agg_df.sort_values(by=["date", "expiration date", "adj strike", "tag"],
                       inplace=True, ignore_index=True)

    if n["tag"] == "complete":
        voi_complete = agg_df
    elif n["tag"] == "incomplete":
        voi_incomplete = agg_df


Processing -> (year: 2005, type: complete)
Processing -> (year: 2006, type: complete)
Processing -> (year: 2007, type: complete)
Processing -> (year: 2008, type: complete)
Processing -> (year: 2009, type: complete)
Processing -> (year: 2010, type: complete)
Processing -> (year: 2011, type: complete)
Processing -> (year: 2012, type: complete)
Processing -> (year: 2013, type: complete)
Processing -> (year: 2014, type: complete)
Processing -> (year: 2015, type: complete)
Processing -> (year: 2016, type: complete)
Processing -> (year: 2017, type: complete)
Processing -> (year: 2018, type: complete)
Processing -> (year: 2019, type: complete)
Processing -> (year: 2020, type: complete)
Processing -> (year: 2021, type: complete)
Processing -> (year: 2019, type: incomplete)
Processing -> (year: 2020, type: incomplete)
Processing -> (year: 2021, type: incomplete)


## Below cells only to see difference between delta open interest and volume

Remove options that have no volume and no change in open interest since they do not tell us anything.

Attach sample from uniform distribution (any works) to allow for random sampling.

In [8]:
voi_complete_lite = pd.DataFrame()

for n in [{"df": voi_complete, "tag": "complete"}]:
    temp_df = n["df"].copy()
    temp_df = temp_df[(temp_df["volume 1"] + temp_df["volume 2"] + temp_df["abs delta"]) != 0]
    temp_df["rand"] = np.random.uniform(0, 1, temp_df.shape[0])
    # This is the randomization step
    temp_df.sort_values(by="rand", inplace=True)
    # So .iloc can be used
    temp_df.reset_index(drop=True, inplace=True)

    if n["tag"] == "complete":
        voi_complete_lite = temp_df


Plot |change in "open interest"| vs. "volume"

We also plot the line `y = x` ($Volume = \Delta Open~Interest$). As mentioned before, volume should always be greater or equal to the change in open interest, never less than (except in the case of option exercise, which shouldn't happen as much as trading). So anything "below" this line can be considered "erroneous".

Only completed options are plotted, as incomplete options would yield similar results.

In [9]:
# Select n samples from each of calls and puts to plot
number_points = 10000

interest_vs_volume_fig = interest_vs_volume(input_dict={"complete": voi_complete_lite.iloc[:number_points, :]})

interest_vs_volume_fig.write_image("./img/EDA1_volume_vs_interest.svg", width=1200, height=600)

![Volume vs delta open interest](../img/EDA1_volume_vs_interest.svg)

Percentage of total points under the line.

In [11]:
error_df = pd.DataFrame()

for year in sorted(set(voi_complete_lite["year"])):
    year_df = voi_complete_lite[voi_complete_lite["year"] == year].copy()

    n1_error_rate = year_df[year_df["abs delta"] > year_df["volume 1"]].shape[0] / year_df.shape[0]
    n2_error_rate = year_df[year_df["abs delta"] > year_df["volume 2"]].shape[0] / year_df.shape[0]

    error_df = error_df.append(pd.DataFrame([[year, n1_error_rate, n2_error_rate]]))

error_df.columns = ["Year", "Day n error rate", "Day n+1 error rate"]

error_df.reset_index(drop=True, inplace=True)

error_df

,Year,Day n error rate,Day n+1 error rate
0,2005,0.137885,0.367895
1,2006,0.051043,0.293278
2,2007,0.039911,0.253286
3,2008,0.047783,0.258958
4,2009,0.047652,0.286984
5,2010,0.043272,0.266920
6,2011,0.018583,0.259888
7,2012,0.037530,0.282349
8,2013,0.034293,0.284740
9,2014,0.050432,0.302680


The error rate when comparing |Delta open interest| with current day's (`n`) volume is much lower than when comparing with next day's (`n+1`)

This implies that open interest is recorded at the **start of day** (if volume is accurate, and not lagged by a day etc.)

Change in open interest is used as indicator of option movement as opposed volume for two reasons
   - Volume does not have direction
   - Volume can be arbitrarily inflated

Note that change in open interest is the sum of market transactions and exercises, the two of which we are unable to distinguish systematically. It could be true that many of the "errors" that occurred when comparing day `n` volume and open interest could be due to this.

## Feature Engineering Cont'd

Using the above conclusion:
   1. Adjust feature "open interest" to reflect "end of day open interest"
   2. Add "delta open interest" feature

Some additional engineered features:
   1. `ask er`: `ask price` * `delta interest`
   2. `sign` - the sign of `delta interest`

In [12]:
# Final Dataframes ("year" to be removed later)
complete_combined_df = pd.DataFrame()
incomplete_combined_df = pd.DataFrame()

for n in [{"df1": complete_df, "df2": voi_complete, "tag": "complete"},
          {"df1": incomplete_df, "df2": voi_incomplete, "tag": "incomplete"}]:
    # Merge
    temp_df = n["df1"].merge(
        n["df2"][["date", "expiration date", "tag", "adj strike", "open interest 2", "delta interest"]],
        how="inner", on=["date", "expiration date", "tag", "adj strike"],
        validate="1:1")

    # Add additional features here
    temp_df["price spread"] = (temp_df["ask price"] - temp_df["bid price"]).round(6)

    temp_df["ask er"] = (temp_df["ask price"] * temp_df["delta interest"]).round(6)

    temp_df["delta sign"] = np.sign(temp_df["delta interest"])

    # Columns to save
    save_columns = ["date", "expiration date", "days till exp", "adj strike",
                    "tag", "ask price", "price spread",
                    "volume", "open interest 2", "delta interest",
                    "adj moneyness", "adj breakeven",
                    "ask er", "delta sign",
                    "adj date close", "year"]

    if n["tag"] == "complete":
        save_columns.append("adj exp close")

    temp_df = temp_df[save_columns]

    temp_df.rename(columns={"open interest 2": "EOD open interest"}, inplace=True)

    temp_df.sort_values(by=["date", "expiration date", "adj strike", "tag"],
                        inplace=True, ignore_index=True)

    if n["tag"] == "complete":
        complete_combined_df = temp_df
    elif n["tag"] == "incomplete":
        incomplete_combined_df = temp_df


## Below cells only to see volume of options vs. time

Visualization of volume and |delta open interest| vs time.

Only done on "complete" options. "Incomplete" should not be statistically different.

**Black vertical lines indicate ex-dividend dates**

In [14]:
# Aggregate delta open interest and volume data
voi_dividends_df = complete_combined_df[["date", "year", "volume", "delta interest"]].copy()
voi_dividends_df['abs delta'] = np.abs(voi_dividends_df["delta interest"])

voi_dividends_df = voi_dividends_df.groupby(["year", "date"])[["volume", "abs delta"]].agg("sum").reset_index()

# Load dividend time series
dividends_df = pd.read_csv(os.path.join(dividends_path, f"{ticker}.csv"))

dividends_df["div start"] = pd.to_datetime(dividends_df["div start"]).dt.date

dividends_df = dividends_df[["div start"]]


We notice that some days have total of 0 |delta open interest|. They are most likely errors in recording open interest.

Step-like fashion of it's value further suggests this.

In [15]:
voi_dividends_df[voi_dividends_df["abs delta"] < 1000]

,year,date,volume,abs delta
346,2006,2006-05-22,2201472.0,0.0
349,2006,2006-05-25,1419460.0,0.0
452,2006,2006-10-20,2467528.0,0.0
874,2008,2008-06-26,7580160.0,0.0


In [16]:
voi_dividends_fig = voi_dividends_ts({"min year": 2005, "max year": 2021,
                                      "voi agg": voi_dividends_df, "dividends": dividends_df})

voi_dividends_fig.write_image("./img/EDA1_voi_dividends_ts.svg", width=1800, height=800)

|Delta open interest| plot uses a minimum y-value of the next highest recording > 0 so that plot isn't too skewed.

![Volume vs delta open interest](../img/EDA1_voi_dividends_ts.svg)

## Export data from above

In [19]:
years_list = sorted(set(complete_combined_df["year"]) | set(incomplete_combined_df["year"]))

for year in years_list:
    year_save_path = os.path.join(save_path, str(year))
    Path(year_save_path).mkdir(parents=True, exist_ok=True)

    temp_complete_df = complete_combined_df[complete_combined_df["year"] == year].copy()
    temp_incomplete_df = incomplete_combined_df[incomplete_combined_df["year"] == year].copy()

    temp_complete_df.drop(columns="year", inplace=True)
    temp_incomplete_df.drop(columns="year", inplace=True)

    if temp_complete_df.shape[0] > 0:
        temp_complete_df.to_csv(
            path_or_buf=os.path.join(save_path, str(year), f"EDA1_{ticker}_{year}_complete.csv"),
            index=False)

    if temp_incomplete_df.shape[0] > 0:
        temp_incomplete_df.to_csv(
            path_or_buf=os.path.join(save_path, str(year), f"EDA1_{ticker}_{year}_incomplete.csv"),
            index=False)
